In [1]:
# import
import os
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch
import gym
from tqdm import tqdm

import pyrootutils

path = pyrootutils.find_root(search_from=os.path.abspath(''), indicator=".project-root")
pyrootutils.set_root(path = path,
                     project_root_env_var = True,
                     dotenv = True,
                     pythonpath = True)
                     
PATH = str(path).replace("\\","/")

from transformer.gpt_transformer.src.utils import D4RLTrajectoryDataset
from transformer.gpt_transformer.src.model import DecisionTransformer


In [2]:
# set hyperparameter
env_name = 'halfcheetah'
dataset = 'medium'

if env_name == 'hopper':
    env = gym.make('Hopper-v2')

elif env_name == 'halfcheetah':
    env = gym.make('HalfCheetah-v2')

elif env_name == 'walker2d':
    env = gym.make('Walker2d-v2')


if torch.cuda.is_available():
    DEVICE = torch.device('cuda:0')
else:
    DEVICE = torch.device('cpu')


c:\Users\zkdlx\miniconda3\envs\rl_diffusion\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment HalfCheetah-v2 is out of date. You should consider upgrading to version `v3` with the environment ID `HalfCheetah-v3`.
  logger.warn(


In [3]:
# check dim

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

print("state dim: ", state_dim)
print("action dim: ", act_dim)

state dim:  17
action dim:  6


In [4]:
# load best model

BEST_MODEL_PATH = f"{PATH}/transformer/gpt_transformer/src/best_model/{env_name}-{dataset}_model_best.pt"

batch_size = 128
embed_dim = 128
activation = 'relu'
drop_out = 0.1
k = 31
n_blocks = 3
n_heads = 1 # transformer head

AUG_DATA_PATH = f'{PATH}/transformer/gpt_transformer/src/data/augmented/{env_name}-{dataset}-v2.npz'
FILTERED_DATA_PATH = f'{PATH}/transformer/gpt_transformer/src/data/filtered/{env_name}-{dataset}-v2.npz'

best_model = DecisionTransformer(
            state_dim=state_dim,
            act_dim=act_dim,
            n_blocks=n_blocks,
            h_dim=embed_dim,
            context_len=k,
            n_heads=n_heads,
            drop_p=drop_out,
        ).to(DEVICE)


# load checkpoint
best_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=DEVICE))
best_model.eval()



DecisionTransformer(
  (transformer): Sequential(
    (0): Block(
      (attention): MaskedCausalAttention(
        (q_net): Linear(in_features=128, out_features=128, bias=True)
        (k_net): Linear(in_features=128, out_features=128, bias=True)
        (v_net): Linear(in_features=128, out_features=128, bias=True)
        (proj_net): Linear(in_features=128, out_features=128, bias=True)
        (att_drop): Dropout(p=0.1, inplace=False)
        (proj_drop): Dropout(p=0.1, inplace=False)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (1): Block(
      (attention): MaskedCausalAttention(
        (q_net): Linear(in_features=128, out_features=12

In [5]:
# load augmented dataset
aug_dataset_sample = np.load(AUG_DATA_PATH, allow_pickle=True)
aug_dataset_sample = aug_dataset_sample['data']

In [6]:
print(aug_dataset_sample[0])

{'observations': array([[-9.78982896e-02,  1.13477379e-01,  1.98929846e-01,
         6.73338473e-02, -2.23132133e-01, -9.95479763e-01,
        -4.97307360e-01, -5.16024351e-01,  9.02984238e+00,
         1.12497166e-01,  4.16288280e+00, -3.49108428e-01,
        -1.01194935e+01, -5.76310039e-01, -8.60520935e+00,
        -7.22706127e+00,  1.51465490e-01],
       [-9.34970081e-02,  1.04090542e-01, -1.35129943e-01,
         2.85682887e-01, -3.97425026e-01, -3.31844419e-01,
        -2.32354969e-01, -4.77415584e-02,  9.32954693e+00,
        -1.71609208e-01, -1.79579675e+00, -8.79514313e+00,
         8.46916866e+00, -4.05881453e+00,  2.01686878e+01,
         1.33829222e+01,  1.36221838e+01],
       [-7.25239292e-02,  3.64939049e-02, -4.39952761e-01,
        -1.81274801e-01, -4.14618701e-01,  4.15624201e-01,
        -1.49758965e-01,  2.10397542e-01,  8.33133316e+00,
         4.72206712e-01, -1.19015479e+00, -3.44378471e+00,
        -1.55777407e+01,  7.04492033e-01,  1.08165216e+01,
         2.6

In [7]:
print("shape of dataset: ", aug_dataset_sample.shape)
print("# of episode: ", len(aug_dataset_sample))
print("content len: ", aug_dataset_sample[0]['observations'].shape[0])

shape of dataset:  (161291,)
# of episode:  161291
content len:  31


In [8]:
cnt = 0

for aug in aug_dataset_sample:
    print(aug['observations'].shape)
    
    if cnt >= 5:
        break
    
    cnt+=1

(31, 17)
(31, 17)
(31, 17)
(31, 17)
(31, 17)
(31, 17)


In [9]:
# calculate mean, std

aug_states, aug_next_states, aug_rewards = [], [], []
for traj in aug_dataset_sample:
    aug_states.append(traj['observations'])
    aug_next_states.append(traj['next_observations'])
    aug_rewards.append(traj['rewards'])
    
# used for input, output normalization
aug_states = np.concatenate(aug_states, axis=0)
# print("state shape: ", states.shape)
aug_state_mean, aug_state_std = np.mean(aug_states, axis=0), np.std(aug_states, axis=0)

aug_next_states = np.concatenate(aug_next_states, axis=0)
aug_next_state_mean, aug_next_state_std = np.mean(aug_next_states, axis=0), np.std(aug_next_states, axis=0)

aug_rewards = np.concatenate(aug_rewards, axis=0)
aug_reward_mean, aug_reward_std = np.mean(aug_rewards, axis=0), np.std(aug_rewards, axis=0)

In [16]:
# check augmented dataset
# dataset[episode][feature][timesteps]
# episode <= 161290, timestep <= 30
ori = aug_dataset_sample[161290]['original_observations']
state = aug_dataset_sample[161290]['observations']
next_state = aug_dataset_sample[161290]['next_observations']
action = aug_dataset_sample[161290]['actions']
reward = aug_dataset_sample[161290]['rewards']

terminals = aug_dataset_sample[100]['terminals']

# print("state: ", state)
# print("next_state: ", next_state)
# print("action: ", action)
print("reward: ", reward)
# print("original: ", ori)
print("terminals: ", terminals)


reward:  [7.784991  8.068654  7.8201    8.098979  7.8025093 7.6428986 7.758953
 8.321434  7.928987  8.273634  8.138268  7.816143  8.042273  8.50186
 8.081312  8.483885  8.218922  8.033504  8.07085   8.640026  8.37447
 8.477856  8.407034  8.294989  8.580523  8.785766  7.848864  7.9064064
 7.9307446 8.201475  8.171015 ]
terminals:  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]


In [8]:
aug_dataset = D4RLTrajectoryDataset(aug_dataset_sample, 1, not_path=True)

aug_data_loader = DataLoader(aug_dataset,
						batch_size=1,
						shuffle=True,
						pin_memory=True,
						drop_last=True)
						
aug_data_iter = iter(aug_data_loader)

timesteps, states, next_states, actions, rewards, traj_mask, terminals = next(aug_data_iter)

timesteps = timesteps.to(DEVICE)	# B x T
states = states.to(DEVICE)			# B x T x state_dim
next_states = next_states.to(DEVICE) # B X T X state_dim
actions = actions.to(DEVICE)		# B x T x act_dim
rewards = rewards.to(DEVICE).unsqueeze(dim=-1) # B x T x 1


In [9]:
print(timesteps.shape)
print(states.shape)
print(actions.shape)
print(rewards.shape)
print(terminals.shape)

torch.Size([1, 1])
torch.Size([1, 1, 17])
torch.Size([1, 1, 6])
torch.Size([1, 1, 1])
torch.Size([1, 1])


In [13]:
next_state_preds, rewards_preds = best_model.forward(
                                                rewards=rewards,
                                                timesteps=timesteps,
                                                states=states,
                                                actions=actions,
                                            )

In [14]:
next_state_preds

tensor([[[-0.0146,  0.1910,  1.0978, -0.5555,  0.4784, -0.2861, -0.0347,
          -0.5686, -0.1602, -0.1830,  0.0157, -1.2909,  0.4794,  0.3535,
          -0.5570, -0.2047,  0.4830]]], grad_fn=<ViewBackward0>)

In [15]:
print("next_state_shape: ", next_state_preds.shape)
print("rewards_shape: ", rewards_preds.shape)

# next_state_preds, rewards_preds

next_state_shape:  torch.Size([1, 1, 17])
rewards_shape:  torch.Size([1, 1, 1])


In [16]:
# filtering
Percentage = 0.1 # 0.1 ~ 1

def filtering_transformer(augmented_dataset_sample, model, Percentage=Percentage):
    
    # temp = 0
    
    filtered_dataset = pd.DataFrame(columns = ['states', 'next_states', 'actions', 'rewards', 'timesteps', 'traj_mask', 'terminals', 'mse'])
    
    states_list, next_states_list, actions_list, rewards_list, timesteps_list, traj_mask_list, terminals_list, mse_list = [], [], [], [], [], [], [], []
    
    aug_dataset = D4RLTrajectoryDataset(augmented_dataset_sample, k, not_path=True)

    aug_data_loader = DataLoader(aug_dataset,
                            batch_size=1,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=True)
                            
    for timesteps, states, next_states, actions, rewards, traj_mask, terminals in tqdm(aug_data_loader):
        
        states_list.append(np.array(states.reshape(k, state_dim)))
        next_states_list.append(np.array(next_states.reshape(k, state_dim)))
        actions_list.append(np.array(actions.reshape(k, act_dim)))
        rewards_list.append(np.array(rewards.reshape(-1,)))
        timesteps_list.append(np.array(np.squeeze(timesteps, axis=0)))
        traj_mask_list.append(np.array(np.squeeze(traj_mask, axis=0)))
        terminals_list.append(np.array(np.squeeze(terminals, axis=0)))

        # normalization
        states = (states - aug_state_mean) / aug_state_std
        next_states = (next_states - aug_next_state_mean) / aug_next_state_std
        rewards = (rewards - aug_reward_mean) / aug_reward_std
        
        timesteps = timesteps.to(DEVICE)	# B x T
        states = states.to(DEVICE)			# B x T x state_dim
        next_states = next_states.to(DEVICE) # B X T X state_dim
        actions = actions.to(DEVICE)		# B x T x act_dim
        rewards = rewards.to(DEVICE).unsqueeze(dim=-1) # B x T x 1
        traj_mask = traj_mask.to(DEVICE)	# B x T
        terminals = terminals.to(DEVICE)	# B x T
    
        pred_next_states = torch.clone(next_states).detach().to(DEVICE)
        pred_rewards = torch.clone(rewards).detach().to(DEVICE)
    
        real_next_state, real_rewards = model.forward(
                                                        rewards=rewards,
                                                        timesteps=timesteps,
                                                        states=states,
                                                        actions=actions,
                                                    )
        pred_next_states = pred_next_states.view(-1, state_dim)[traj_mask.view(-1,) > 0]
        real_next_state = real_next_state.view(-1, state_dim)[traj_mask.view(-1,) > 0]

        pred_rewards = pred_rewards.view(-1, 1)[traj_mask.view(-1,) > 0]
        real_rewards = real_rewards.view(-1, 1)[traj_mask.view(-1,) > 0]

        state_loss = F.mse_loss(pred_next_states, real_next_state, reduction='mean')
        reward_loss = F.mse_loss(pred_rewards, real_rewards, reduction='mean')
        
        total_loss = state_loss.add(reward_loss)
        total_loss = torch.mean(total_loss)
        mse_list.append(total_loss.detach().cpu().item())
        
        # temp += 1
        
        # if temp == 10:
        #     break
                                                    
    filtered_dataset['states'] = states_list
    filtered_dataset['next_states'] = next_states_list
    filtered_dataset['actions'] = actions_list
    filtered_dataset['rewards'] = rewards_list
    filtered_dataset['timesteps'] = timesteps_list
    filtered_dataset['traj_mask'] = traj_mask_list
    filtered_dataset['terminals'] = terminals_list
    filtered_dataset['mse'] = mse_list
    
    filtered_dataset.sort_values(by='mse', ascending=True, inplace=True)
    
    print("# of augmented dataset: ", len(filtered_dataset))
    keep_rows = int(len(filtered_dataset) * (1-Percentage))
    
    filtered_dataset = filtered_dataset.head(keep_rows)
    filtered_dataset = filtered_dataset.sample(frac=1).reset_index(drop=True)
    print("# of filtered dataset: ", len(filtered_dataset))
    
    # dataframe to numpy array with dict
    np_filtered_dataset = []
    
    for i in range(len(filtered_dataset)):
        np_filtered_dataset.append({'observations': np.array(filtered_dataset['states'][i]), 
                                    'next_observations': np.array(filtered_dataset['next_states'][i]),
                                    'actions': np.array(filtered_dataset['actions'][i]),
                                    'rewards': np.array(filtered_dataset['rewards'][i]),
                                    'timesteps': np.array(filtered_dataset['timesteps'][i]),
                                    'traj_mask': np.array(filtered_dataset['traj_mask'][i]),
                                    'terminals': np.array(filtered_dataset['terminals'][i]),
                                    'mse': np.array(filtered_dataset['mse'][i]),
                                    })
        
    
    return np_filtered_dataset
    


In [17]:
filtered_dataset = filtering_transformer(aug_dataset_sample, best_model, Percentage=Percentage)

100%|██████████| 161291/161291 [22:02<00:00, 121.97it/s] 


# of augmented dataset:  161291
# of filtered dataset:  145161


In [18]:
filtered_dataset[0]['observations'].shape

(31, 17)

In [19]:
# save filtered dataset -> .npz

temp_array = np.array([1,2,])

np.savez(FILTERED_DATA_PATH, data=filtered_dataset, config=temp_array)

In [20]:
# load filtered dataset

filtered_dataset_sample = np.load(FILTERED_DATA_PATH, allow_pickle=True)
filtered_dataset_sample = filtered_dataset_sample['data']

In [21]:
# check filtered dataset

# filtered_data[epi][feature][timestep]

print("# of dataset: ", len(filtered_dataset_sample))
print("state_shape: ", filtered_dataset_sample[0]['observations'].shape)
print("1 episode: ", filtered_dataset_sample[0])
print("state: ", filtered_dataset_sample[0]['observations'])

# of dataset:  145161
state_shape:  (31, 17)
1 episode:  {'observations': array([[-5.95069453e-02, -1.34868383e-01,  4.90116477e-02,
         8.92765522e-02, -4.30434406e-01, -3.67374063e-01,
        -8.88491943e-02, -5.38930535e-01,  7.36041212e+00,
        -3.71103197e-01,  2.00792694e+00,  4.35801888e+00,
         1.54553471e+01,  1.19976294e+00, -1.47329082e+01,
        -3.50908613e+00, -3.71288967e+00],
       [-8.05613101e-02,  7.61705711e-02,  6.66418970e-02,
         2.62775123e-02, -3.15470964e-01, -1.05718255e+00,
        -4.45620775e-01, -4.30622935e-01,  7.08609915e+00,
        -2.80249268e-01,  4.88628435e+00, -2.75292397e+00,
        -8.08739376e+00,  1.63108981e+00, -1.03430882e+01,
        -7.58965397e+00,  2.88419580e+00],
       [-7.39868656e-02,  1.24384038e-01, -1.63010001e-01,
        -1.43998250e-01, -3.86974454e-01, -3.78842533e-01,
        -2.41812259e-01,  2.81131789e-02,  7.79753304e+00,
        -6.24826029e-02, -5.60067654e-01, -5.72627497e+00,
        -1.422